# Suivi de l'apprentissage dans KERAS: sauvegarde automatique du meilleur modèle et mécanisme de "Early Stopping"



Les **callbacks** Keras sont des mécanismes permettant d'intéragir avec l'étape d'apprentissage du modèle. Ils permettent notamment:

1. de stocker les performances mesurées lors de l'apprentissage via le callback "**CSVLogger**"
2. de suivre intéractivement l'évolution de ces performance via le callback "**TensorBoard**"
3. d'implémenter un mécanisme de régularisation basée sur le "**EarlyStopping**" (via le callback du même nom)
4. d'enregistrer automatiquement le meilleur modèle obtenu le long des "epochs" de l'apprentissage, via le callback "**ModelCheckpoint**".

En pratique, on les spécifie lors de l'appel à la fonction "**fit()**" du modèle.



Dans ce notebook nous allons voir comment **mettre en oeuvre les callbacks CSVLogger, ModelCheckpoint et EarlyStopping**.
Nous illustrerons leur intérêt pour **éviter le risque de sur-apprentissage observé lors du TP précédent pour l'apprentissage d'un perception multi-couches** sur les données MNIST.




On commence par charger les packages et par bloquer la graine du générateur aléatoire par souci de reproductibilité. 
**Attention : keras faisant partie de tensorflow depuis la version 2 de tensorflow, les imports ne seront pas les mêmes si vous travaillez à partir de la version 2 ou d'une version antérieure.**
Pour cela, renseigner la variable **"tensorflow_version"** dans la cellule ci-dessous à la valeur **"V1"** ou **"V2"** en fonction de votre configuration.

In [ ]:
# specify tensorflow version
tensorflow_version = "V2"

In [ ]:
# import packages
import csv
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import pickle
# import tensorflow requirements - depending on version
if tensorflow_version == "V1":
    from keras import backend as K
    from keras.models import Model
    from keras.models import Sequential
    from keras.models import load_model
    from keras.layers import Dense, Activation, Flatten
    from keras.layers import Conv2D, MaxPooling2D
    from keras.utils import to_categorical
    from keras.callbacks import ModelCheckpoint
    from keras.callbacks import CSVLogger
    from keras.callbacks import EarlyStopping
if tensorflow_version == "V2":
    from tensorflow.keras.models import Model
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.models import load_model
    from tensorflow.keras.layers import Dense, Activation, Flatten
    from tensorflow.keras.layers import Conv2D, MaxPooling2D
    from tensorflow.keras.utils import to_categorical
    from tensorflow.keras.callbacks import ModelCheckpoint
    from tensorflow.keras.callbacks import CSVLogger
    from tensorflow.keras.callbacks import EarlyStopping
# set random seed
np.random.seed(123)

#  Préparation des données

Nous commençons par charger le jeu de données sauvegardé par le premier notebook du TP précédent.

In [ ]:
f = open('mnist-dataset.pickle', 'rb')
dataset = pickle.load(f)
f.close()

In [ ]:
X_train = dataset[0]
X_test  = dataset[1]
Y_train = dataset[2]
Y_test  = dataset[3]
y_train = dataset[4]
y_test  = dataset[5]

# Réseaux de neurones multicouches & sur-apprentissage

On reproduit ensuite le phénomène de sur-apprentissage observé lors du TP précédent, en apprenant un perceptron multi-couche à deux couches cachées pendant 100 epochs. 

Avant cela, il nous faut néanmoins **transformer les données originales** (des images) pour en avoir une représentation vectorielle, comme on l'a fait pour la régression multinomiale. On commence donc par transformer chaque image de taille 28x28 en un vecteur de taille 784,

In [ ]:
input_dim = 28*28 
X_train_vec = X_train.reshape(60000, input_dim) 
X_test_vec  = X_test.reshape(10000, input_dim)

In [ ]:
# define multi-layer perceptron with two hidden layers of 64 and 100 neurons to adress the multiclass problem
mlp = Sequential()
mlp.add(Dense(64, activation='relu', input_dim=input_dim))
mlp.add(Dense(100, activation='relu'))
mlp.add(Dense(10, activation='softmax'))
mlp.summary()

In [ ]:
mlp.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy']) 
history_mlp = mlp.fit(X_train_vec, Y_train, batch_size=32, epochs=100, verbose=1, validation_split=0.2)

In [ ]:
plt.figure(figsize=(8,4))
# show accuracy
plt.subplot(1,2,1)
plt.plot(history_mlp.history['accuracy'], label = 'train')
plt.plot(history_mlp.history['val_accuracy'], label = 'validation')
plt.title("accuracy vs epochs")
plt.legend()
# show loss
plt.subplot(1,2,2)
plt.plot(history_mlp.history['loss'], label = 'train')
plt.plot(history_mlp.history['val_loss'], label = 'validation')
plt.title("loss vs epochs")
plt.legend()
plt.show()

# EXERCICE: Implémenter un mécanisme de callback pour stocker automatiquement le meilleur modèle obtenu lors de l'apprentissage. En profiter pour créer un second callback visant à enregistrer les performances mesurées lors de l'apprentissage dans un fichier texte (.csv). 

On utilisera pour cela les callback **ModelCheckPoint** et **CSVLogger**.

**On n'oubliera pas de ré-initialiser le modèle pour que l'apprentissage reprenne depuis le début.**

On ré-initialise le modèle.

In [ ]:
# re-create model to re-initialize parmaters 
mlp = Sequential()
mlp.add(Dense(64, activation='relu', input_dim=input_dim))
mlp.add(Dense(100, activation='relu'))
mlp.add(Dense(10, activation='softmax'))
mlp.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy']) 

On "re-fitte" le modèle avec 100 epochs en ajoutant des **"callbacks"** pour sauvegarder le meilleur modèle retenu lors de l'apprentissage  et sauvegarder les performances dans un fichier texte.

In [ ]:
# TODO: define callback to save best model
checkPointModel = ModelCheckpoint( ... )

# TODO: define callback to save log file
csv_logger = CSVLogger( ... )

# fit with callbacks
history_mlp = mlp.fit(X_train_vec, Y_train, batch_size=32, epochs=100, verbose=1, 
                      validation_split=0.2, callbacks = [checkPointModel, csv_logger])

# EXERCICE: Charger le modèle retenu à l'aide de la fonction load_model() et s'en servir pour analyser le jeu de test.

In [ ]:
# load model 
best_mlp = load_model( ... )

In [ ]:
score_mlp = mlp.evaluate(X_test_vec, Y_test, verbose=0)
score_mlp_best = best_mlp.evaluate(X_test_vec, Y_test, verbose=0)
print("multi-layer perceptron test-set accuracy =", score_mlp[1])
print("best multi-layer perceptron test-set accuracy =", score_mlp_best[1])

# EXERCICE:  Combiner tout cela avec un callback "EarlyStopping". Considérer une "patience" de 5 epochs. Au bout de combien d'epoch l'apprentissage s'arrête t'il ?  

On utilisera pour cela les callback **EarlyStopping**.

**Là aussi, on n'oubliera pas de ré-initialiser le modèle pour que l'apprentissage reprenne depuis le début.**

On ré-initialise le modèle.

In [ ]:
# re-create model to re-initialize parmaters 
mlp = Sequential()
mlp.add(Dense(64, activation='relu', input_dim=input_dim))
mlp.add(Dense(100, activation='relu'))
mlp.add(Dense(10, activation='softmax'))
mlp.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy']) 

In [ ]:
# TODO : define callback to save best model
checkPointModel = ModelCheckpoint( ... )

# TODO : define callback to save log file
csv_logger = CSVLogger( ... )

# define early stoppin callback
early_stop = EarlyStopping( ... )

# fit with callbacks
history_mlp = mlp.fit(X_train_vec, Y_train, batch_size=32, epochs=100, verbose=1, 
                      validation_split=0.2, callbacks = [checkPointModel, csv_logger, early_stop])

On représente l'evolution de l'"historique" de l'apprentissage (i.e., l'évolution de l'accuracy et de la loss mesurées sur les données d'apprentissage et de validation) pour mesurer l'impact du "early stopping".

In [1]:
# TODO : plot evolution of "history" (i.e., train + validation loss and accuracy) to assess impact of early stopping